In [1]:
import os
from sagemaker.huggingface.model import HuggingFaceModel
from sagemaker.async_inference.async_inference_config import AsyncInferenceConfig
from sagemaker.s3 import s3_path_join
from sagemaker.async_inference.waiter_config import WaiterConfig

import pandas as pd
import awswrangler as wr
import numpy as np


import threading
import time
import matplotlib.pyplot as plt

#SageMaker Role
ROLE = os.getenv('SAGEMAKER_ROLE')
# Hub Model configuration. https://huggingface.co/models
hub = {
    'HF_MODEL_ID':'unitary/unbiased-toxic-roberta',
    'HF_TASK':'text-classification'
}
# sagemaker_session_bucket = 'sagemaker-godeltech'

In [2]:
# create Hugging Face Model Class
huggingface_model = HuggingFaceModel(
    transformers_version='4.17.0',
    pytorch_version='1.10.2',
    py_version='py38',
    env=hub,
    role=ROLE
)

# deploy model to SageMaker Inference
realtime_predictor = huggingface_model.deploy(
        initial_instance_count=1, # number of instances
        instance_type='ml.m5.xlarge', # ec2 instance type
        endpoint_name=f"toxicbert-huggingdace-realtime"
)

-----!

In [3]:
##let's try on kayne west scandal tweet https://pitchfork.com/news/kanye-west-locked-out-of-twitter-following-anti-semitic-tweet/

kayne_west_tweet = """
I’m a bit sleepy tonight but when I wake up I’m going death con 3 On JEWISH PEOPLE.
The funny thing is I actually can’t be Anti Semitic because black people are actually Jew 
also You guys have toyed with me and tried to black ball anyone whoever opposes your agenda.
"""

In [4]:
%%time

realtime_predictor.predict({"inputs" : kayne_west_tweet})

CPU times: user 78.1 ms, sys: 0 ns, total: 78.1 ms
Wall time: 865 ms


[{'label': 'jewish', 'score': 0.9979433417320251}]

In [5]:
# create async endpoint configuration
async_config = AsyncInferenceConfig(
    output_path=s3_path_join("s3://", sagemaker_session_bucket, "transformers/async_predict/outputs") , # Where our results will be stored
)

# deploy the endpoint endpoint
async_predictor = huggingface_model.deploy(
    initial_instance_count=1,
    instance_type="ml.m5.xlarge",
    async_inference_config=async_config,
    endpoint_name=f"toxicbert-huggingdace-asynch"

)

-----!

In [6]:
async_predictor.predict({"inputs" : kayne_west_tweet})

[{'label': 'jewish', 'score': 0.9979433417320251}]

In [7]:
resp = async_predictor.predict_async({"inputs" : kayne_west_tweet})

config = WaiterConfig(
  max_attempts=5, #  number of attempts
  delay=10 #  time in seconds to wait between attempts
  )

resp.get_result(config)

[{'label': 'jewish', 'score': 0.9979433417320251}]

In [13]:
N_TIMES = 1_000
TEST_PATH = f"s3://{sagemaker_session_bucket}/data/test/test.csv"
test = wr.s3.read_csv([TEST_PATH]).iloc[:N_TIMES, 0]

In [18]:
output_list = []
for text in test:
    resp = async_predictor.predict_async(data={"inputs": text})
    output_list.append(resp)

print(f"Response object: {resp}")
print(f"Response output path: {resp.output_path}")
print("Start Polling to get response:")

config = WaiterConfig(
  max_attempts=5, #  number of attempts
  delay=10 #  time in seconds to wait between attempts
  )

resp.get_result(config)

Response object: <sagemaker.async_inference.async_inference_response.AsyncInferenceResponse object at 0x7f5fa6ecf0a0>
Response output path: s3://sagemaker-godeltech/transformers/async_predict/outputs/e55b3128-a8af-40d2-9872-26cfa409b527.out
Start Polling to get response:


[{'label': 'white', 'score': 0.9930591583251953}]

In [19]:
realtime_predictor.delete_model()
realtime_predictor.delete_endpoint()
async_predictor.delete_model()
async_predictor.delete_endpoint()

Sagemaker endpoints *are not publicly exposed* to the Internet. So, you'll need some way of creating a public HTTP endpoint that can route requests to your Sagemaker endpoint. One way you can do this is with an AWS Lambda function fronted by API gateway.

>API Gateway -> Lambda -> Sagemaker endpoint

Basic example is here https://aws.amazon.com/blogs/machine-learning/call-an-amazon-sagemaker-model-endpoint-using-amazon-api-gateway-and-aws-lambda/